# Experiments to get json files loaded into SQL table

In [1]:
cat sql.py

#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

from pyspark.rdd import RDD, PipelinedRDD
from pyspark.serializers import BatchedSerializer, PickleSerializer

from py4j.protocol import Py4JError

__all__ = ["SQLContext", "HiveContext", "LocalHiveContext", "TestHi

In [2]:
from pyspark.sql import *

In [3]:
import tempfile, shutil

In [4]:
from array import array
from pyspark.sql import SQLContext

In [6]:
sc = SparkContext

In [7]:
sqlCtx = SQLContext(sc)

AttributeError: type object 'SparkContext' has no attribute '_jsc'

In [23]:
srdd = sqlCtx.jsonFile('jsonFile')

In [26]:
srdd.collect() == [{"f1": 1, "f2": "row1", "f3":{"field4":11}},
                    {"f1": 2, "f2": "row2", "f3":{"field4":22}},
                    {"f1": 3, "f2": "row3", "f3":{"field4":33}}]

True

#https://www.mail-archive.com/commits@spark.apache.org/msg01502.html

In [25]:
from array import array
from pyspark.sql import SQLContext

In [21]:
sqlCtx = SQLContext(sc)
rdd = sc.parallelize([
        {"f1" : array('i', [1, 2]), "f2" : {"row1" : 1.0}},
        {"f1" : array('i', [2, 3]), "f2" : {"row2" : 2.0}}])

In [22]:
srdd = sqlCtx.inferSchema(rdd)
srdd.collect() == [{"f1" : array('i', [1, 2]), "f2" : {"row1" : 1.0}},
                   {"f1" : array('i', [2, 3]), "f2" : {"row2" : 2.0}}]

True

In [23]:
rdd = sc.parallelize([
        {"f1" : [[1, 2], [2, 3]], "f2" : set([1, 2]), "f3" : (1, 2)},
        {"f1" : [[2, 3], [3, 4]], "f2" : set([2, 3]), "f3" : (2, 3)}])

In [26]:
srdd = sqlCtx.inferSchema(rdd)
srdd.collect() == [{"f1" : [[1, 2], [2, 3]], "f2" : set([1, 2]), "f3" : (1, 2)},
                   {"f1" : [[2, 3], [3, 4]], "f2" : set([2, 3]), "f3" : (2, 3)}]

True

# Load Spark SQL from JSON files

In [29]:
jsonFile
!cat 'jsonFile'

{"f1": 1, "f2": "row1", "f3":{"field4":11}}
{"f1": 2, "f2": "row2", "f3":{"field4":22}}
{"f1": 3, "f2": "row3", "f3":{"field4":33}}


In [15]:
jsonStrings = !cat jsonFile

In [16]:
ofn = open(jsonFile, 'w')

In [17]:
for json in jsonStrings:
    print>>ofn, json

In [18]:
json

'{"f1": 3, "f2": "row3", "f3":{"field4":33}}'

In [19]:
ofn.close()
!cat '/var/folders/dj/92mp96m54d90mdpqlmwbz1m40000gn/T/tmpMFXhlP'

{'f1': 1, 'f2': 'row1', 'f3': {'field4': 11}}
{'f1': 2, 'f2': 'row2', 'f3': {'field4': 22}}
{'f1': 3, 'f2': 'row3', 'f3': {'field4': 33}}


In [20]:
srdd = sqlCtx.jsonFile(jsonFile)

In [22]:
srdd.collect() == [{"f1": 1, "f2": "row1", "f3":{"field4":11}},
                    {"f1": 2, "f2": "row2", "f3":{"field4":22}},
                    {"f1": 3, "f2": "row3", "f3":{"field4":33}}]

True

#http://people.apache.org/~pwendell/spark-1.0.1-rc1-docs/sql-programming-guide.html

In [30]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [33]:
lines = sc.textFile("../../examples/src/main/resources/people.txt")
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: {"name": p[0], "age": int(p[1])})


In [34]:
schemaPeople = sqlContext.inferSchema(people)
schemaPeople.registerAsTable("people")

In [35]:
teenagers = sqlContext.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")


In [36]:
teenNames = teenagers.map(lambda p: "Name: " + p.name)
for teenName in teenNames.collect():
  print teenName

Name: Justin


In [37]:
!cat ../../examples/src/main/resources/people.txt 

Michael, 29
Andy, 30
Justin, 19
